# 目的
#每日收盤後開始爬股價

#可以視情況條種是否抓營收，大概都隔月的10 後營收才會公布
## 【資料源】
來自 _BaseInfo  _CrawBase 會先去取基本相關資料 (股票號碼...等)

## 【輸出】
股票相關推薦圖表/數據

In [1]:
import pandas as pd
import threading
from datetime import datetime, timedelta

In [2]:
#%run _BaseInfoETF.ipynb
%run _CrawBase.ipynb
#%run _LineMsg.ipynb
%run _genHtml.ipynb
#%run RunRealTimeStock.ipynb

C:\Users\user\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [3]:
def MainStock(stock_number, start_month,end_time=datetime.now().strftime("%Y-%m-%d"),updateRevenue=False):  

    strmsg ='【stock_number】'+str(stock_number)+'\n' +'【start_month】'+str(start_month)+'\n'+'【end_time】'+str(end_time)+'\n' 
    try:
        #處理 Stock
        RowData_df_craw_stock,His_Stock ,isSuccess= craw_stock(stock_number,start_month,end_time)
        #data_final,df_final=DataCleaning(RowData_df_craw_stock)  
        
        #處理 Revenue
        if(updateRevenue):
            RowData_df_craw_revenue,His_Stock=craw_revenue(stock_number)
            df_Revenue=RevenueDataCleaning(RowData_df_craw_revenue)
            
        #處理相關指標轉成html 
        stock_data=data_process(RowData_df_craw_stock)
        level,interval_type, lower_bound, upper_bound=get_interval(stock_data.iloc[-1]['收盤價'],np.array(stock_data['收盤價']))
        stock_data['stock_number']=stock_number
        stock_data['level']=level
        stock_data['interval_type']=interval_type
        stock_data['lower_bound']=lower_bound
        stock_data['upper_bound']=upper_bound
        
        #print('sendLineMessage')
        stock_data['年月日'] = stock_data['年月日'].astype(str).str.replace('*', '', regex=False)
        stock_data['年月日'] = pd.to_datetime(stock_data['年月日'], format='%Y-%m-%d')
  
        save_to_df(stock_data.iloc[-1].to_frame().T)
        
        gen_html(stock_number,stock_data)
        #sendLineMessage('成功'+'\n'+strmsg )
    except Exception as error:
        #sendLineMessage('失败'+'\n'+strmsg)
        print('MainStock Fail'+'\n'+strmsg)
        print(f"An exception occurred with MainStock {stock_number}: {error}")

In [4]:
start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False

# 定義一個處理股票資料的函數，接收股票代號列表作為參數
def process_stock_codes(stock_codes):
    # 計算月份字串（僅需計算一次）
    strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
    for code in stock_codes:
        stock_number = str(code)
        try:
            MainStock(stock_number, start_month,datetime.now().strftime("%Y-%m-%d"), want_update_Revenue)
        except Exception as error:
            print(f"An exception occurred with stock {stock_number}: {error}")
            
def daily_craw_stock():
    
    # 分割資料，取前 500 個和其餘的股票代號
    first_500_codes = _baseInfo[_baseInfo['Type']=='上市'].dropna()['公司代號']
    remaining_codes =_baseInfo[_baseInfo['Type']!='上市'].dropna()['公司代號']

    # 創建兩個執行緒，分別處理前 500 個和其餘股票代號
    thread1 = threading.Thread(target=process_stock_codes, args=(first_500_codes,))
    thread2 = threading.Thread(target=process_stock_codes, args=(remaining_codes,))

    # 啟動執行緒
    thread1.start()
    thread2.start()

    # 等待兩個執行緒完成
    thread1.join()
    thread2.join()
    
    #save_data_by_date(df_save_old_RunRealTimeStock,"Result/DailyRunRealTimeStock")
    save_data_by_date(df_save_df_to_excel,"Result/DailyFiles")

    print("Stock processing completed.")
    

In [ ]:
daily_craw_stock()

# 補資料 

today =datetime.now().strftime("%Y-%m-%d")
df_info=_baseInfo.copy()
fullpath = f'D:\\Project\\Jupyter\\Stock\\Main\\Result\\DailyRunRealTimeStock\\{today}.xlsx'
my=pd.read_excel(fullpath)

df_info['公司代號']=df_info['公司代號'].astype(str)
my['stock_number']=my['stock_number'].astype(str)
# 找出只在 df_info 中存在，但不在 my 中的 stock_number
aa=df_info[~df_info['公司代號'].isin(my['stock_number'])]
#stock_codes=list(set(aa.dropna()['公司代號']))




# 分割資料，取前 500 個和其餘的股票代號
first_500_codes = aa[aa['Type']=='上市'].dropna()['公司代號']
remaining_codes =aa[aa['Type']!='上市'].dropna()['公司代號']

# 創建兩個執行緒，分別處理前 500 個和其餘股票代號
thread1 = threading.Thread(target=process_stock_codes, args=(first_500_codes,))
thread2 = threading.Thread(target=process_stock_codes, args=(remaining_codes,))

# 啟動執行緒
thread1.start()
thread2.start()

# 等待兩個執行緒完成
thread1.join()
thread2.join()

save_data_by_date(df_save_old_RunRealTimeStock,"Result/DailyRunRealTimeStock")
save_data_by_date(df_save_df_to_excel,"Result/DailyFiles")

print("Stock processing completed.")